In [ ]:
# ============================================================
# 📘 PROYECTO: Predicción de Desempeño en las Pruebas Saber Pro
# ============================================================
# 🧾 Overview:
# Las Pruebas Saber Pro son exámenes estandarizados aplicados en Colombia
# para evaluar la calidad de la educación superior.
# Evalúan cinco componentes: Inglés, Lectura Crítica, Competencias Ciudadanas,
# Razonamiento Cuantitativo y Comunicación Escrita.
#
# 🎯 Objetivo:
# Crear un modelo de clasificación que prediga el desempeño de un estudiante
# (bajo, medio-bajo, medio-alto o alto) a partir de sus características.
#
# En este notebook se muestra cómo se carga el archivo `train.csv`,
# se limpian los datos y se realiza el preprocesamiento necesario.
# ============================================================

# 1️⃣ IMPORTAR LIBRERÍAS
# ------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import csv

# 2️⃣ CARGAR ARCHIVO train.csv
# ------------------------------------------------------------
# Detección automática del separador (por si el CSV usa ";" en lugar de ",")
with open("train.csv", 'r', encoding='utf-8') as f:
    sample = f.read(2048)
    try:
        dialect = csv.Sniffer().sniff(sample)
        sep = dialect.delimiter
    except Exception:
        sep = ','

print(f"🧭 Separador detectado: '{sep}'")

train = pd.read_csv("train.csv", sep=sep, low_memory=False)
print("✅ Archivo cargado correctamente.")
print("Forma del dataset:", train.shape)
display(train.head())

# 3️⃣ EXPLORACIÓN INICIAL
# ------------------------------------------------------------
print("\n📊 Información general:")
print(train.info())

print("\n🔍 Porcentaje de valores nulos:")
display(train.isnull().mean().sort_values(ascending=False).head(10))

print("\n📈 Estadísticas descriptivas:")
display(train.describe(include='all').T)

# 4️⃣ LIMPIEZA BÁSICA
# ------------------------------------------------------------
# Eliminar columnas innecesarias o identificadores
cols_to_drop = [c for c in ['id', 'ID'] if c in train.columns]
if cols_to_drop:
    train.drop(columns=cols_to_drop, inplace=True)
    print(f"🧹 Columnas eliminadas: {cols_to_drop}")

# Eliminar duplicados
duplicados = train.duplicated().sum()
if duplicados > 0:
    train.drop_duplicates(inplace=True)
    print(f"🧹 Se eliminaron {duplicados} filas duplicadas.")
else:
    print("✅ No hay duplicados en el dataset.")

# Eliminar columnas vacías
train.dropna(axis=1, how='all', inplace=True)

# 5️⃣ DETECTAR VARIABLES NUMÉRICAS Y CATEGÓRICAS
# ------------------------------------------------------------
num_cols = train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print("\n📈 Variables numéricas:", len(num_cols))
print("🧩 Variables categóricas:", len(cat_cols))

# 6️⃣ DEFINIR PIPELINES DE PREPROCESADO
# ------------------------------------------------------------
# - Para numéricas: imputar con la mediana y normalizar
# - Para categóricas: imputar valores faltantes y codificar con OneHotEncoder

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='__missing__')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinar ambos pipelines
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# 7️⃣ APLICAR TRANSFORMACIONES
# ------------------------------------------------------------
X_processed = preprocessor.fit_transform(train)
print("\n✅ Transformación completada correctamente.")
print("Dimensiones del dataset transformado:", X_processed.shape)

# 8️⃣ CREAR NOMBRES DE COLUMNAS RESULTANTES
# ------------------------------------------------------------
num_features = num_cols
cat_features = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(cat_cols)
final_columns = np.concatenate([num_features, cat_features])

# Crear DataFrame final
train_processed = pd.DataFrame(X_processed, columns=final_columns)
print("\n✅ DataFrame final creado.")
print("Tamaño:", train_processed.shape)
display(train_processed.head())

# 9️⃣ GUARDAR RESULTADOS
# ------------------------------------------------------------
train_processed.to_csv("train_processed.csv", index=False)
joblib.dump(preprocessor, "preprocessor.joblib")

print("\n💾 Archivos generados correctamente:")
print("- train_processed.csv  → Dataset limpio y listo para modelar")
print("- preprocessor.joblib  → Pipeline de preprocesamiento guardado")

# 🔟 CONCLUSIÓN
# ------------------------------------------------------------
print("""
🏁 Proceso completado exitosamente.
Se ha realizado:
- Carga y revisión del dataset `train.csv`
- Limpieza de columnas vacías, duplicadas y no relevantes
- Imputación de valores faltantes
- Escalado de variables numéricas
- Codificación One-Hot de variables categóricas
- Exportación de dataset procesado y pipeline
""")


🧭 Separador detectado: ','
✅ Archivo cargado correctamente.
Forma del dataset: (272814, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294



📊 Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272814 entries, 0 to 272813
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           272814 non-null  int64  
 1   PERIODO_ACADEMICO            272814 non-null  int64  
 2   E_PRGM_ACADEMICO             272814 non-null  object 
 3   E_PRGM_DEPARTAMENTO          272813 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  270332 non-null  object 
 5   E_HORASSEMANATRABAJA         260689 non-null  object 
 6   F_ESTRATOVIVIENDA            260231 non-null  object 
 7   F_TIENEINTERNET              262380 non-null  object 
 8   F_EDUCACIONPADRE             263651 non-null  object 
 9   F_TIENELAVADORA              257200 non-null  object 
 10  F_TIENEAUTOMOVIL             255690 non-null  object 
 11  E_PRIVADO_LIBERTAD           272813 non-null  object 
 12  E_PAGOMATRICULAPROPIO        27025

,0
F_TIENEAUTOMOVIL,0.062768
F_TIENELAVADORA,0.057233
F_TIENECOMPUTADOR,0.054818
F_ESTRATOVIVIENDA,0.046123
E_HORASSEMANATRABAJA,0.044444
F_TIENEINTERNET.1,0.038246
F_TIENEINTERNET,0.038246
F_EDUCACIONMADRE,0.034177
F_EDUCACIONPADRE,0.033587
E_PAGOMATRICULAPROPIO,0.009398



📈 Estadísticas descriptivas:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,272814.0,NaN,NaN,NaN,494618.513779,285442.540862,11.0,247622.75,494578.5,741484.5,989281.0
PERIODO_ACADEMICO,272814.0,NaN,NaN,NaN,20198.367232,10.530992,20183.0,20195.0,20195.0,20203.0,20213.0
E_PRGM_ACADEMICO,272814,924,DERECHO,21049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E_PRGM_DEPARTAMENTO,272813,31,BOGOTÁ,111090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E_VALORMATRICULAUNIVERSIDAD,270332,8,Entre 1 millón y menos de 2.5 millones,80299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E_HORASSEMANATRABAJA,260689,5,Más de 30 horas,98334,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F_ESTRATOVIVIENDA,260231,7,Estrato 2,92153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F_TIENEINTERNET,262380,2,Si,233324,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F_EDUCACIONPADRE,263651,12,Secundaria (Bachillerato) completa,50864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F_TIENELAVADORA,257200,2,Si,221934,NaN,NaN,NaN,NaN,NaN,NaN,NaN


🧹 Columnas eliminadas: ['ID']
✅ No hay duplicados en el dataset.

📈 Variables numéricas: 5
🧩 Variables categóricas: 15

✅ Transformación completada correctamente.
Dimensiones del dataset transformado: (272814, 1036)

✅ DataFrame final creado.
Tamaño: (272814, 1036)


,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,F_EDUCACIONMADRE___missing__,RENDIMIENTO_GLOBAL___missing__,RENDIMIENTO_GLOBAL_alto,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,1.294540,0.438545,-0.559018,0.813828,0.057959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.294540,0.348286,-0.483954,0.507265,0.013715,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.439918,0.233411,-0.494678,0.728672,0.013715,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.319746,1.776018,-0.945057,-0.173985,-1.077631,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.294540,0.389313,-0.301658,0.388047,0.456152,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0



💾 Archivos generados correctamente:
- train_processed.csv  → Dataset limpio y listo para modelar
- preprocessor.joblib  → Pipeline de preprocesamiento guardado

🏁 Proceso completado exitosamente.
Se ha realizado:
- Carga y revisión del dataset `train.csv`
- Limpieza de columnas vacías, duplicadas y no relevantes
- Imputación de valores faltantes
- Escalado de variables numéricas
- Codificación One-Hot de variables categóricas
- Exportación de dataset procesado y pipeline

